# Q7
Tree Based Models - Q07- 08/July

Consider the dataset in credit_data.csv at the following location https://drive.google.com/drive/folders/1Jl8iDu7nGmrqCECbrLqmVafgwE5PYfiU

A1 - A15 = Attributes, T - Target (positive or negative credit)

Split the data into train and test ( 75% - 25%) and train a decision tree using sklearn DecisionTreeClassifier() with 2 different methods

- 1) ID3  (criterion = 'entropy')
- 2) CART (criterion = 'gini')

Find the accuracy on test data for the 2 different methods. Why do you think they are so?

In [1]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

from sklearn import tree
pd.set_option('display.max_columns', None)

import warnings
warnings.filterwarnings("ignore")

In [2]:
# Read and display the data file
credit_data = pd.read_csv('/Users/riteshturlapaty/ai-ml-learning/AccelerateAI/7. DecisionTree/DailyQuiz/01_credit_data.csv')
credit_data

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,T
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,f,g,202,0,+
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6,f,g,43,560,+
2,a,24.5,0.500,u,g,q,h,1.50,t,f,0,f,g,280,824,+
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5,t,g,100,3,+
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,f,s,120,0,+
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
685,b,21.08,10.085,y,p,e,h,1.25,f,f,0,f,g,260,0,-
686,a,22.67,0.750,u,g,c,v,2.00,f,t,2,t,g,200,394,-
687,a,25.25,13.500,y,p,ff,ff,2.00,f,t,1,t,g,200,1,-
688,b,17.92,0.205,u,g,aa,v,0.04,f,f,0,f,g,280,750,-


In [3]:
credit_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 690 entries, 0 to 689
Data columns (total 16 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   A1      690 non-null    object 
 1   A2      690 non-null    object 
 2   A3      690 non-null    float64
 3   A4      690 non-null    object 
 4   A5      690 non-null    object 
 5   A6      690 non-null    object 
 6   A7      690 non-null    object 
 7   A8      690 non-null    float64
 8   A9      690 non-null    object 
 9   A10     690 non-null    object 
 10  A11     690 non-null    int64  
 11  A12     690 non-null    object 
 12  A13     690 non-null    object 
 13  A14     690 non-null    object 
 14  A15     690 non-null    int64  
 15  T       690 non-null    object 
dtypes: float64(2), int64(2), object(12)
memory usage: 86.4+ KB


In [4]:
# Check A2 as it looks numeric but data type is object
credit_data['A2'].value_counts(normalize=True)*100

?        1.739130
22.67    1.304348
20.42    1.014493
18.83    0.869565
24.5     0.869565
           ...   
48.25    0.144928
28.33    0.144928
18.75    0.144928
18.5     0.144928
36.42    0.144928
Name: A2, Length: 350, dtype: float64

In [5]:
# Dorp the rows having values ? as their proportion is only 1.7%
credit_data.drop(credit_data[credit_data['A2'] == '?'].index, inplace=True)

In [6]:
# Check A14 as it looks numeric but data type is object
val = credit_data['A14'].value_counts(normalize=True)*100
val[:20]

0      19.026549
120     5.162242
200     5.014749
160     5.014749
100     4.424779
80      4.424779
280     3.244838
180     2.654867
140     2.359882
240     2.064897
320     1.917404
?       1.769912
300     1.769912
260     1.622419
400     1.327434
60      1.327434
220     1.327434
340     1.032448
360     1.032448
440     0.589971
Name: A14, dtype: float64

In [7]:
# Dorp the rows having values ? as their proportion is only 1.7%
credit_data.drop(credit_data[credit_data['A14'] == '?'].index, inplace=True)

In [8]:
# Check shape
credit_data.shape

(666, 16)

In [9]:
credit_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 666 entries, 0 to 689
Data columns (total 16 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   A1      666 non-null    object 
 1   A2      666 non-null    object 
 2   A3      666 non-null    float64
 3   A4      666 non-null    object 
 4   A5      666 non-null    object 
 5   A6      666 non-null    object 
 6   A7      666 non-null    object 
 7   A8      666 non-null    float64
 8   A9      666 non-null    object 
 9   A10     666 non-null    object 
 10  A11     666 non-null    int64  
 11  A12     666 non-null    object 
 12  A13     666 non-null    object 
 13  A14     666 non-null    object 
 14  A15     666 non-null    int64  
 15  T       666 non-null    object 
dtypes: float64(2), int64(2), object(12)
memory usage: 88.5+ KB


In [10]:
# Lets convert A2 to float and A14 to int64
credit_data['A2'] = credit_data['A2'].astype('float')
credit_data['A14'] = credit_data['A14'].astype('int64')

In [11]:
# Segregate column names as per their datatype i.e this is to convert the values into numeric
cat_var_list = credit_data[['A1', 'A4', 'A5', 'A6', 'A7', 'A9', 'A10', 'A12', 'A13']]
num_var_list = credit_data[['A2', 'A3', 'A8', 'A11', 'A14', 'A15']]
target_list=credit_data[['T']]

In [12]:
cat_var_list

,A1,A4,A5,A6,A7,A9,A10,A12,A13
0,b,u,g,w,v,t,t,f,g
1,a,u,g,q,h,t,t,f,g
2,a,u,g,q,h,t,f,f,g
3,b,u,g,w,v,t,t,t,g
4,b,u,g,w,v,t,f,f,s
...,...,...,...,...,...,...,...,...,...
685,b,y,p,e,h,f,f,f,g
686,a,u,g,c,v,f,t,t,g
687,a,y,p,ff,ff,f,t,t,g
688,b,u,g,aa,v,f,f,f,g


In [13]:
num_var_list

,A2,A3,A8,A11,A14,A15
0,30.83,0.000,1.25,1,202,0
1,58.67,4.460,3.04,6,43,560
2,24.50,0.500,1.50,0,280,824
3,27.83,1.540,3.75,5,100,3
4,20.17,5.625,1.71,0,120,0
...,...,...,...,...,...,...
685,21.08,10.085,1.25,0,260,0
686,22.67,0.750,2.00,2,200,394
687,25.25,13.500,2.00,1,200,1
688,17.92,0.205,0.04,0,280,750


In [14]:
# Create dummy values for columns where values are non-numeric
cat_var_dummies=pd.get_dummies(cat_var_list)

In [15]:
cat_var_dummies

,A1_?,A1_a,A1_b,A4_l,A4_u,A4_y,A5_g,A5_gg,A5_p,A6_?,A6_aa,A6_c,A6_cc,A6_d,A6_e,A6_ff,A6_i,A6_j,A6_k,A6_m,A6_q,A6_r,A6_w,A6_x,A7_?,A7_bb,A7_dd,A7_ff,A7_h,A7_j,A7_n,A7_o,A7_v,A7_z,A9_f,A9_t,A10_f,A10_t,A12_f,A12_t,A13_g,A13_p,A13_s
0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,1,0,1,0,0
1,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,1,0,1,0,0
2,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,1,0,1,0,0
3,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,1,1,0,0
4,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,1,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
685,0,0,1,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,1,0,1,0,0
686,0,1,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,1,1,0,0
687,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,1,1,0,0
688,0,0,1,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,1,0,1,0,0


In [16]:
# Convert target variable values into numeric. Map the values
target_list['T'] = target_list['T'].map({'+':1, '-':0})

# Check T
target_list['T'].value_counts()

0    367
1    299
Name: T, dtype: int64

In [17]:
num_var_list.shape

(666, 6)

In [18]:
cat_var_dummies.shape

(666, 43)

In [19]:
new_credit_card = pd.concat([num_var_list,cat_var_dummies], axis=1)

In [20]:
new_credit_card

,A2,A3,A8,A11,A14,A15,A1_?,A1_a,A1_b,A4_l,A4_u,A4_y,A5_g,A5_gg,A5_p,A6_?,A6_aa,A6_c,A6_cc,A6_d,A6_e,A6_ff,A6_i,A6_j,A6_k,A6_m,A6_q,A6_r,A6_w,A6_x,A7_?,A7_bb,A7_dd,A7_ff,A7_h,A7_j,A7_n,A7_o,A7_v,A7_z,A9_f,A9_t,A10_f,A10_t,A12_f,A12_t,A13_g,A13_p,A13_s
0,30.83,0.000,1.25,1,202,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,1,0,1,0,0
1,58.67,4.460,3.04,6,43,560,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,1,0,1,0,0
2,24.50,0.500,1.50,0,280,824,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,1,0,1,0,0
3,27.83,1.540,3.75,5,100,3,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,1,1,0,0
4,20.17,5.625,1.71,0,120,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,1,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
685,21.08,10.085,1.25,0,260,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,1,0,1,0,0
686,22.67,0.750,2.00,2,200,394,0,1,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,1,1,0,0
687,25.25,13.500,2.00,1,200,1,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,1,1,0,0
688,17.92,0.205,0.04,0,280,750,0,0,1,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,1,0,1,0,0


In [21]:
# Prepare X and y
X=new_credit_card
y=target_list

In [22]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.75, random_state=42)

In [23]:
# ID3
clf = tree.DecisionTreeClassifier(criterion='entropy')
clf = clf.fit(X_train, y_train)

In [24]:
# Predit on test data
p_pred = clf.predict_proba(X_test)
y_pred = clf.predict(X_test)

score_ = clf.score(X_test, y_test)
print(score_)

0.8323353293413174


In [25]:
# CART
clf2 = tree.DecisionTreeClassifier(criterion='gini')
clf2 = clf2.fit(X_train, y_train)

In [26]:
# Predict on test data
p_pred = clf2.predict_proba(X_test)
y_pred = clf2.predict(X_test)

score_ = clf2.score(X_test, y_test)
print(score_)

0.7844311377245509


ID3 score: 0.8323353293413174
CART score: 0.7844311377245509

As seen, ID3 score is marginal better than CARD but not substantial hence any of the model is preferred